# امیرمهدی کوششی ۹۸۱۷۱۰۵۳
# محمد صادق مجیدی ۹۸۱۰۶۰۰۴
# امیرحسین باقری ۹۸۱۰۵۶۲۱

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=5>
در این تمرین قصد داریم که وسیله نقلیه را در متن پیدا کنیم اگر وسیله نقلیه در متن پیدا شد مبدا و مقصد را نیز پیدا می‌کنیم.
            </font>
</div>

## الگوریتم حل

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
ابندا قصد داشتیم که مبدا ها و مقصد هارا نیز با استفاده از دیتاست شهر ها پیدا کنیم اما به این نکته برخوردیم که هزاران هزار ترکیب اسمی می‌توانند مبدا و مقصد باشند بنابراین تصمیم بر آن شد که 
از تگر هضم استفاده کنیم. و گروه های اسمی را به عنوان مبدا و مقصد اعلام کنیم.
            </font>
</div>

### استفاده از هضم

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
با توجه به مثال های داده شده در داک تمرین مبدا و مقصد و وسیله نقلیه را در یک جمله پیدا می‌کنیم.
برای جمله بندی رشته داده شده از ابزار هضم استفاده می‌کنیم.
            </font>
</div>


```c
    def get_analysis(self,token, mode = 0):
        assert 0 <= mode < 5 ,"0  <= mode < 5"
        if mode == 0:
            return self.lemmatizer.lemmatize(token)
        if mode == 1:
            return self.stemmer.stem(token)
        if mode == 2:
            return self.normalizer.normalize(token)
        if mode == 3:
            return sent_tokenize(self.normalizer.normalize(token))
        if mode == 4:
            return word_tokenize(token)
```

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
بنابراین برای جمله بندی از خود هضم استفاده می‌کنیم.
جملات را نرمالایز نیز میکنیم که از غلط های ویرایشی بکاهیم.
            </font>
</div>

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=5>
دقت کنید که برای تعیین کردن 
span بازه های مشخص شده اسپن نسبت به جملات تعیین میگردد 
که فرض معقولی می ‌باشد (‌با توجه به داک اگر بخواهیم اسپن کلی را بدست آوریم کافیت اسپن فعلی را با طول جملات قبلی جمع کنیم اما توجه دارید که این کار واقعا نیاز نیست و بخاطر مثال های درون داک تمرین این کار را انجام نمیدهیم و اسپن بازه های خواسته شده را از ابتدای جمله تشخیص داده شده توسط هضم بیان می‌کنیم.)
            </font>
</div>

### لود کردن موارد مورد نیاز

```c
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()
        self.stemmer = Stemmer()
        self.tagger = POSTagger(model='./resources/postagger.model')
        self.chunker = Chunker(model='resources/chunker.model')
        self.vehicles = {s : True  for s in pd.read_csv("resources/vehicles_final.csv")["vehicles"].unique()}
        self.places = {s : True  for s in pd.read_csv("resources/places.csv")["place"].unique()}
```

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
تگر و چانکر و همچنین دیگر موارد هضم را لود میکنیم همچنین برای استفاده رجکس های مورد نیاز برای تشخصی مبدا و مقصد را لود می‌کنیم
همچنین یک رجکس از دیتا ست وسایل نقلیه ایجاد می‌کنیم زیرا در قسمت های بعدی نیاز است.
            </font>
</div>
<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
فرمت رجکس ها به شرح زیر است <br>
به + مقصد
<br>
(افعال نظیر آمدم رفتم)‌ + مقصد
<br>
از + مبدا
<br>
برای پیدا کردن NP آنها ابتدا تمام ‌‌NP ها را با کمک هضم پیدا می‌کنیم سپس اسپن آنهارا بدست‌ می‌اوریم 
حال که اسپن NP هارا داریم 
اسپن اولین کلمه پیدا شده برای مقصد یا مبدا را پیدا می‌کنیم.
این کار به کمک رجکس های شرح داده شده صورت می‌گیرد.
سپس میبینیم که آن اسپن در کدامین اسپن NP ها  می‌گنجند.
آن NP را انتخاب می‌کنیم.
            </font>
</div>
<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
پیدا کردن وسایل نقلیه <br>
از ‌NP های پیدا شده بررسی می‌کنیم که کدام یک طولانی ترین وسیله نقلیه درون دیتاست را داراست سپس آن وسیله نقلیه را بررسی می‌کنیم که آیا قبل از آن با آمده است یا نه 
زیرا رجکس وسیله نقلیه بصورت پیش فرض زیر است
<br>
با + وسیله نقلیه 
<br>
اما از آنجا که هدف اصلی پیدا کردن وسیله نقلیه است وسایل نقلیه ای که قبل آنها با نیست را نیز نگه می‌داریم.
            </font>
</div>

# الگوریتم اصلی

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
حال به توضیح تابع run می پردازیم.
دقت نمایید که به توضیخ خط به خط نمی‌پردازیم و تنها ایده های هر کیس را مطرح می‌کنیم
ابتدا توضیح می‌دهیم که چه فرایندی روی جمله صورت می‌پذیرد.
            </font>
</div>

```c
sentence = self.get_analysis(sen, 4)  
chunked = self.chunker.parse(self.tagger.tag(sentence))
tree = tree2brackets(chunked)
NP_W = self.find_NP(tree)
NP_W_span=self.find_spans(NP_W,sen)
posibble_vehicles = self.find_vehicles(NP_W)
checked_vehicles,spans,posibble_vehicles = self.check_is_vehicle_after_ba(posibble_vehicles, sen)
```
<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
این کار روی تمام جملات صورت میگیرد در پایان وسایلی که قبل آنها با امده است با اسپن هر یک از NP ها را داریم 
همچنین ذکر این نکته نیز حاءز اهمیت است که اگر با رجکس با حرف اضافه به مقصدی پیدا نشود سراغ افعال می‌رویم و مقصد را بدان شکل پیدا می‌کنیم.
حال به توضیح هر کیس می پردازیم.
<ol>
  <li>وسیله نقلیه که قبل آن با باشد پیدا شود: در این حالت وسیله نقلیه را قرار داده و مقصد و مبدا را پیدا می‌کنیم. </li>
  <li>وسیله نقلیه قبل آن با نباشد در این حالت وسیله نقلیه را مقداری ریلکس تر برخورد کرده و اگر مبدا و مقصدی پیدا شد فرض می‌کنیم همان وسیله بوده است .
  دقت کنید ممکن است مبدا و مقصدی پیدا نشود.
  </li>
  <li>اگر در ‌NP ها وسیله نقلیه پیدا نشود در این صورت دست به دامان رجکس شده و اولین وسیله نقلیه پیدا شده توسط رجکس را به عنوان وسیله نقلیه قرار داده و مبدا و مقصد را در صورت وجود پیدا می‌کنیم.</li>
</ol>
            </font>
</div>

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=4>
اگر وسیله نقلیه پیدا نشود دیگر مبدا و مقصدی نیز پیدا نمی‌کنیم.
            </font>
</div>

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=6>
ماژول خواسته شده ماژول در فایل زیر است.
            </font>
</div>

```c
Vehicles.py
Vehicle_detector

from Vehicles import Vehicle_detector
```

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=6>
چند مثال
            </font>
</div>

In [10]:
from Vehicles import Vehicle_detector
vehicle_detector = Vehicle_detector()
print("case 1 : ")
print("فردا با فولکس از تهران به اصفهان می‌روم")
print(vehicle_detector.run("فردا به فولکس از تهران به اصفهان می‌روم."))
print("از جنگل های شمال با تراکتور آمدم شهر.")
print(vehicle_detector.run("از جنگل های شمال با تراکتور آمدم شهر."))
print("از کشور بیگانگان با هواپیما با چشمانی پر خون به مام وطن باز گشت.")
print(vehicle_detector.run("از کشور بیگانگان با هواپیما با چشمانی پر خون به مام وطن باز گشت."))

print("case 2 : ")
print("کشتی مسافران را از بندر امام خمینی به بندر دبی برد.")
print(vehicle_detector.run("کشتی مسافران را از بندر امام خمینی به بندر دبی برد."))

print("case 3 : ")
print("نیرو های امدادی توسط هلیکوپترشان مادربزرگم را از حیاط پشتی به بیمارستان بردند.")
print(vehicle_detector.run("نیرو های امدادی توسط هلیکوپترشان مادربزرگم را از حیاط پشتی به بیمارستان بردند."))

print("case 4")
print(vehicle_detector.run("از شمال شیراز به جنوب اصفهان رفتم."))



loading model ... 
loading tagger ... 
loading chuncker ... 
reading dataset ... 
loading regex ... 
loading completed
case 1 : 
فردا با فولکس از تهران به اصفهان می‌روم
[{'from': 'تهران', 'from_span': (17, 22), 'to': 'فولکس', 'to_span': (8, 13), 'vehicle': 'فولکس', 'vehicle_span': (8, 13)}]
از جنگل های شمال با تراکتور آمدم شهر.
[{'from': 'جنگل های شمال', 'from_span': (3, 16), 'to': 'شهر', 'to_span': (33, 36), 'vehicle': 'تراکتور', 'vehicle_span': (20, 27)}]
از کشور بیگانگان با هواپیما با چشمانی پر خون به مام وطن باز گشت.
[{'from': 'کشور بیگانگان', 'from_span': (3, 16), 'to': 'مام وطن', 'to_span': (48, 55), 'vehicle': 'هواپیما', 'vehicle_span': (20, 27)}]
case 2 : 
کشتی مسافران را از بندر امام خمینی به بندر دبی برد.
[{'from': 'بندر امام خمینی', 'from_span': (19, 34), 'to': 'بندر دبی', 'to_span': (38, 46), 'vehicle': 'کشتی', 'vehicle_span': (0, 4)}]
case 3 : 
نیرو های امدادی توسط هلیکوپترشان مادربزرگم را از حیاط پشتی به بیمارستان بردند.
[{'from': 'حیاط پشت', 'from_span': (49, 57), 'to': 

In [2]:
#%%writefile Vehicles.py
from __future__ import unicode_literals
import pandas as pd
from hazm import *
import pandas as pd
import re
from termcolor import colored
import pandas as pd
from hazm import *
import pandas as pd
import re
from termcolor import colored
class Vehicle_detector:
    def __init__(self):
        self.mode = True
        print(colored("loading model ... ","blue"))
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()
        self.stemmer = Stemmer()
        print(colored("loading tagger ... ","blue"))
        self.tagger = POSTagger(model='./resources/postagger.model')
        print(colored("loading chuncker ... ","blue"))
        self.chunker = Chunker(model='resources/chunker.model')
        print(colored("reading dataset ... ","blue"))
        self.vehicles = {s : True  for s in pd.read_csv("resources/vehicles_final.csv")["vehicles"].unique()}
        self.places = {s : True  for s in pd.read_csv("resources/places.csv")["place"].unique()}
        f = open("resources/dest_verbs.txt","r",encoding = "utf-8")
        self.dest_verbs = [l.strip() for l in f.readlines()]
        f.close()
        print(colored("loading regex ... ","blue"))
        self.NP_regex = "\[[^\[]* NP\]"
        self.VP_regex = "\[[^\[]* VP\]"
        self.After_ba = "(\\S+)?" + "( |\u200c)"
        self.BA = "با"
        self.SPAN_finder = "(\\S+)?" + "(" + " " + "|" + "‌" + ")"
        self.AZ_source = "از (\\S+)( )?"
        self.BE_dest = "به (\\S+)( )?"
        self.FIND_BY_verb = "("
        size = len(self.dest_verbs)-1
        for verb in range(size):
            v = self.dest_verbs[verb].strip()
            self.FIND_BY_verb = self.FIND_BY_verb+ v+"|"
        self.FIND_BY_verb = self.FIND_BY_verb+self.dest_verbs[size]+") "+"(\\S+)"
        self.VEHICLE_REGEX = "( )?("
        for key in self.vehicles:
            self.VEHICLE_REGEX = self.VEHICLE_REGEX + key+"|"
        self.VEHICLE_REGEX = self.VEHICLE_REGEX[0:-1]+")( |شان|تان|مان|اش|ام|ت|م|ش)?"
        print(colored("loading completed","blue"))

    def get_analysis(self,token, mode = 0):
        assert 0 <= mode < 5 ,"0  <= mode < 5"
        if mode == 0:
            return self.lemmatizer.lemmatize(token)
        if mode == 1:
            return self.stemmer.stem(token)
        if mode == 2:
            return self.normalizer.normalize(token)
        if mode == 3:
            return sent_tokenize(self.normalizer.normalize(token))
        if mode == 4:
            return word_tokenize(token)
    def ste_lem(self,sentence , mode = 0):
        assert 0 <= mode <= 1 ,"mode 1 or 0"
        return [self.get_analysis(s,1-mode) for s in sentence]

    def find_NP(self,chunked_tree):
        res = re.findall(self.NP_regex, chunked_tree)
        return [" ".join([self.get_analysis(w) for w in word[1:-4].strip().replace("\u200c", " ").split()]).strip() for word in res]

    def find_VP(self,chunked_tree):
        res = re.findall(self.VP_regex, chunked_tree)
        return [" ".join([self.get_analysis(w) for w in word[1:-4].strip().replace("\u200c", " ").split()]).strip() for word in res]

    def find_vehicles_each(self,word):
        if self.vehicles.get(word):
            return word
        else:
            False
    def set_mode(self,mode):
        self.mode = mode

    def take_apart_words(self,sen):
        candidate_words = []
        for i in sen:
            if len(i.split()) >= 1:
                strr = ""
                splitted_words = i.split()
                longest_accepted_word = ""
                for j in splitted_words:
                    strr += j
                    if self.find_vehicles_each(strr):
                        longest_accepted_word = strr
                    strr += " "
                if longest_accepted_word:
                    candidate_words.append(longest_accepted_word.strip())
        return candidate_words

    def check_is_vehicle_after_ba(self,vehicles, sen):
        checked_vehicles = []
        spans = []
        possible = []
        for vehicle in vehicles:
            vehicle_parts = vehicle.split()
            regex = self.BA+" "
            size = len(vehicle_parts)-1
            for index,part in enumerate(vehicle_parts):
                regex += (part + self.After_ba) if  index != size else part
            regex = regex.strip()
            x = re.search(regex, sen)
            if x:
                start, end = x.span()
                checked_vehicles.append(vehicle)
                spans.append((start+3,end))
            x = re.search(regex[3:], sen)
            if x:
                start, end = x.span()
            possible.append((vehicle,start,end))
            
        return checked_vehicles,spans,possible 

    def find_vehicles(self,sen):
        words = self.take_apart_words(sen)
        if len(words) == 0:
            pass
        return words

    def find_spans(self,NP_Ws, sen: str):
        copy_sen = sen
        base = 0
        spans = []
        for NP_W in NP_Ws:
            NP_W_parts = NP_W.split(" ")
            regex = ""
            size = len(NP_W_parts)-1
            for index,part in enumerate(NP_W_parts):# "(\\S+)?" + "(" + " " + "|" + "‌" + ")"
                regex += (part + self.SPAN_finder) if index != size else part
            
            regex = regex.strip()
            x = re.search(regex, copy_sen)
            start, end = x.span()
            spans.append((NP_W, start + base, end + base))
            copy_sen=copy_sen[end:]
            base = base + end
        return(spans)    

    def check_places(self,sen, NP_W_span):
        sources = []
        destinations = []
        source = re.search(self.AZ_source, sen)
        if source:
            posibble_source_start, posibble_source_end = source.span(1)
            for NP_W in NP_W_span:
                if (NP_W[1] <= posibble_source_start <= posibble_source_end <= NP_W[2] or NP_W[1] == posibble_source_start):
                    sources.append(NP_W)
                    break

        destination = re.search(self.BE_dest, sen)
        if destination:
            posibble_destination_start, posibble_destination_end = destination.span(1)
            for NP_W in NP_W_span:
                if (NP_W[1] <= posibble_destination_start <= posibble_destination_end <= NP_W[2] or NP_W[1] == posibble_destination_start):
                    destinations.append(NP_W)
                    break
        if len(destinations) == 0:
            destination = re.search(self.FIND_BY_verb, sen)
            if destination:
                s, e = destination.span(2)
                if sen[e-1] == ".":
                    e = e-1
                destinations.append((sen[s:e],s,e)) #TODO check by amirmahdi
        if len(destinations) == 0:
            destinations = [("",-1,-1)]
        if len(sources) == 0:
            sources = [("",-1,-1)]
        return sources[0], destinations[0] 
    def create_output(self,source = "",s_span = (-1,-1),dest = "",dest_span = (-1,-1),vehicle = "",vehicle_span = (-1,-1)):
        return {
            "from":source,
            "from_span":s_span,
            "to":dest,
            "to_span":dest_span,
            "vehicle":vehicle,
            "vehicle_span":vehicle_span
        }

    def run(self,input:str):
        outputs = []
        for ind, sen in enumerate(self.get_analysis(input, 3)): 
            output = {} 
            try:
                sentence = self.get_analysis(sen, 4)  
                chunked = self.chunker.parse(self.tagger.tag(sentence))
                tree = tree2brackets(chunked)
                NP_W = self.find_NP(tree)
                NP_W_span=self.find_spans(NP_W,sen)
                posibble_vehicles = self.find_vehicles(NP_W)
                checked_vehicles,spans,posibble_vehicles = self.check_is_vehicle_after_ba(posibble_vehicles, sen)
                if len(checked_vehicles) > 0: #TODO check by amir mahdi 
                    v = checked_vehicles[0]
                    v_s = spans[0]
                    source, destination =self.check_places(sen,NP_W_span)
                    output = self.create_output(
                        source=source[0],
                        s_span=(source[1],source[2]),
                        dest=destination[0],
                        dest_span=(destination[1],destination[2]),
                        vehicle=v,
                        vehicle_span=v_s
                    )
                elif len(posibble_vehicles) > 0:
                    v = posibble_vehicles[0][0]
                    v_s = (posibble_vehicles[0][1],posibble_vehicles[0][2])
                    if self.mode:
                        source, destination =self.check_places(sen,NP_W_span)
                        output = self.create_output(
                            source=source[0],
                            s_span=(source[1],source[2]),
                            dest=destination[0],
                            dest_span=(destination[1],destination[2]),
                            vehicle=v,
                            vehicle_span=v_s
                        )
                    else:
                        output = self.create_output(
                            vehicle=v,
                            vehicle_span=v_s
                        )
                else:
                    output = self.create_output()
                    v = re.search(self.VEHICLE_REGEX, sen) #TODO
                    if v:
                        s, e = v.span(2)
                        if s > 2 and sen[s-3:s-1] == self.BA and e-s > 2:
                            source, destination =self.check_places(sen,NP_W_span)
                            output = self.create_output(
                            source=source[0],
                            s_span=(source[1],source[2]),
                            dest=destination[0],
                            dest_span=(destination[1],destination[2]),
                            vehicle=sen[s:e],
                            vehicle_span=(s,e)
                            )
                        elif e-s > 1:
                            if self.mode:
                                source, destination =self.check_places(sen,NP_W_span)
                                output = self.create_output(
                                source=source[0],
                                s_span=(source[1],source[2]),
                                dest=destination[0],
                                dest_span=(destination[1],destination[2]),
                                vehicle=sen[s:e],
                                vehicle_span=(s,e)
                                )
                            else:
                                output = self.create_output(
                                vehicle=sen[s:e],
                                vehicle_span=(s,e)
                                )
            except:
                output = self.create_output()
            outputs.append(output)

        return outputs


In [3]:
import random
from Vehicles import Vehicle_detector
vehicle_detector = Vehicle_detector()
sample = [line.strip() for line in open("input.txt","r",encoding="utf-8").readlines()]
for s in random.sample(sample,4):
    print("--------------------------------")
    print(f"{s}")
    print(vehicle_detector.run(s))
    print("\n")

loading model ... 
loading tagger ... 
loading chuncker ... 
reading dataset ... 
loading regex ... 
loading completed
--------------------------------
از تهران با دویست و شش برادر دوستم به خونه عموی پدرم رفتیم.
[{'from': 'تهران', 'from_span': (3, 8), 'to': 'خونه', 'to_span': (38, 42), 'vehicle': 'دویست و شش', 'vehicle_span': (12, 22)}]


--------------------------------
با پراید دنده اتومات پدرم به خونه برادر عموم رفتیم.
[{'from': '', 'from_span': (-1, -1), 'to': 'خونه برادر عموم', 'to_span': (29, 44), 'vehicle': 'پراید', 'vehicle_span': (3, 8)}]


--------------------------------
پدرم با پراید آمد اصفهان.
[{'from': '', 'from_span': (-1, -1), 'to': 'اصفهان', 'to_span': (18, 24), 'vehicle': 'پراید', 'vehicle_span': (8, 13)}]


--------------------------------
از بندرعباس تا شمال را با دوچرخه رکاب زدیم.
[{'from': 'بندرعباس', 'from_span': (3, 11), 'to': '', 'to_span': (-1, -1), 'vehicle': 'دوچرخه', 'vehicle_span': (26, 32)}]




In [2]:
%%writefile Vehicles_with_test.py
from Vehicles import Vehicle_detector
vehicle_detector = Vehicle_detector()
f = open("input.txt","r",encoding= "utf-8")
final = ""
for index,line in enumerate(f.readlines()):
    if index == 35:
        print(index)
    else:
        print(index)
    res = vehicle_detector.run(line.strip())
    for a in res:
        s = {
            "index":index,
            "input str" : line.strip(),
            "result" : a,
            "detected_vehicle":a["vehicle"],
            "v_span":a["vehicle_span"],
            "tested_V_span":line[a["vehicle_span"][0]:a["vehicle_span"][1]],
            "detected_from":a["from"],
            "s_span":a["from_span"],
            "tested_S_span":line[a["from_span"][0]:a["from_span"][1]],
            "detected_to":a["to"],
            "d_span":a["to_span"],
            "tested_D_span":line[a["to_span"][0]:a["to_span"][1]],
        }
        final+="\n".join([f"{str(key)} : {s[key]}" for key in s])+"\n"
    final+="\n\n"
f.close()
f = open("res.txt","w",encoding="utf-8")
f.write(final)
f.close()


Overwriting Vehicles_with_test.py
